In [1]:
import goatools
import seaborn as sns
import pandas as pd 
import pickle

# Load Data

In [2]:
df_all_merged_kmeans_data_file = "../data/go_terms/df_all_merged_kmeans_data.p"
df_all_merged_birch_data_file = "../data/go_terms/df_all_merged_birch_data.p"

df_all_merged_kmeans_data = pickle.load(open(df_all_merged_kmeans_data_file, "rb"))
df_all_merged_birch_data = pickle.load(open(df_all_merged_birch_data_file, "rb"))

In [21]:
kmeans_19_file = "../data/cluster_by_loci/kmeans, param1 = 19, param2 = nan, param3 = nan"

df_kmeans_labels = pd.read_csv(kmeans_19_file).fillna('')

## Visualize Imported Dataframes

- Data represents each locus, its expression under the different conditions, go_terms, and metadata regarding the locus itself

In [3]:
df_all_merged_kmeans_data

,locus_tag,cluster_id,LanzaTech,MeOH,NO3_lowO2_slow_growth,NoCu,NoLanthanum,WT_control,WithLanthanum,aa3_KO,...,unknown,gene,product,start_coord,end_coord,length,translation,group,go_terms,desc_string
0,EQU24_RS00005,12,-0.515808,-0.012058,-0.480048,-0.163349,-0.648447,-0.037469,-0.491013,-0.410117,...,-0.620522,dnaA,chromosomal replication initiator protein DnaA,0,1317,1318,MSALWNNCLAKLENEISSSEFSTWIRPLQAIETDGQIKLLAPNRFV...,else,"[GO:0003688, GO:0005524, GO:0005737, GO:000627...",EQU24_RS00005|dnaA|chromosomal replication ini...
1,EQU24_RS00010,4,0.480568,-0.659004,-0.218643,0.075821,-0.485571,-0.114821,-0.462526,0.094641,...,-0.798028,,DNA polymerase III subunit beta,1502,2603,1102,MKYIINREQLLVPLQQIVSVIEKRQTMPILSNVLMVFRENTLVMTG...,else,"[GO:0003677, GO:0003887, GO:0005737, GO:000626...",EQU24_RS00010||DNA polymerase III subunit beta
2,EQU24_RS00015,17,-1.072637,0.327816,-0.461533,-0.212764,-0.583017,-0.648530,-0.633792,-0.917224,...,-0.753895,recF,DNA replication/repair protein RecF,3060,4140,1081,MSLQKLDIFNVRNIRQASLQPSPGLNLIYGANASGKSSVLEAIFIL...,else,"[GO:0003697, GO:0005524, GO:0005737, GO:000626...",EQU24_RS00015|recF|DNA replication/repair prot...
3,EQU24_RS00020,4,0.723618,-1.032390,0.072132,-0.253222,-0.352283,0.183488,-0.292974,0.144541,...,-0.532597,gyrB,DNA topoisomerase (ATP-hydrolyzing) subunit B,4185,6600,2416,MSENIKQYDSTNIQVLKGLDAVRKRPGMYIGDTDDGTGLHHMVFEV...,else,"[GO:0003677, GO:0003918, GO:0005524, GO:000569...",EQU24_RS00020|gyrB|DNA topoisomerase (ATP-hydr...
4,EQU24_RS00025,4,0.424262,-0.479325,0.028098,0.116538,-0.261659,0.082847,-0.454102,-0.051229,...,-0.011128,,hypothetical protein,6825,7062,238,VKTTKYFLTTRMRPDREIIKDEWIQYVVRFPENEHIQFDGRIRRWA...,else,[],EQU24_RS00025||hypothetical protein
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4208,EQU24_RS22135,9,-0.445389,-0.627550,-1.170665,-0.396450,-1.877416,-0.819438,-1.715711,-0.966683,...,-0.784262,mnmE,tRNA uridine-5-carboxymethylaminomethyl(34) sy...,4994941,4996288,1348,VDIGTNDTIAAIATPPGNGGVGIVRISGPAVSIIAAKLTDRPLPPR...,else,"[GO:0003924, GO:0005515, GO:0005525, GO:000573...",EQU24_RS22135|mnmE|tRNA uridine-5-carboxymethy...
4209,EQU24_RS22140,9,0.309593,-1.327737,-1.065740,-0.706776,-1.812366,-0.818297,-1.749368,-0.654022,...,-1.159766,yidC,membrane protein insertase YidC,4996295,4997993,1699,MDNIRFILVVTFAMLLYMLYDAWQIDYGPKREISVAQQMPQDLKED...,else,"[GO:0005886, GO:0015031, GO:0016021, GO:003297...",EQU24_RS22140|yidC|membrane protein insertase ...
4210,EQU24_RS22145,0,-1.028264,-0.395724,-1.265702,-0.657315,-2.492508,-0.803695,-2.140667,-0.926349,...,-1.151259,yidD,membrane protein insertion efficiency factor YidD,4997998,4998220,223,MRVLLIAIIKLYQYFISPLLGKNCRFYPSCSCYALEALHKHGAAQG...,else,[GO:0005886],EQU24_RS22145|yidD|membrane protein insertion ...
4211,EQU24_RS22150,0,-0.670633,-0.279756,-1.404703,-0.775605,-2.384838,-1.209018,-2.238072,-1.143556,...,-1.159678,rnpA,ribonuclease P protein component,4998201,4998570,370,LTNKVFSFPPQLRLRKPSEYKKVFTGPVKSSDAYFTLLAVRNELDH...,else,"[GO:0000049, GO:0001682, GO:0004526, GO:0090502]",EQU24_RS22150|rnpA|ribonuclease P protein comp...


In [76]:
df_all_merged_birch_data['cluster_id']

0       14
1       14
2        3
3       14
4       21
        ..
4208     9
4209     9
4210     9
4211     9
4212     9
Name: cluster_id, Length: 4213, dtype: int64

In [77]:
# For each cluster in this clustering, a set of unique go terms will be developed
def get_go_terms_for_clustering(df_clustering):

    clusters = range(max(df_clustering['cluster_id']) + 1)
    cluster_to_go_term = {}

    for cluster_id in clusters:

        x = df_clustering[df_clustering['cluster_id']== cluster_id]
        go_terms_in_cluster = []
        for group in x['go_terms']: 
            for single in group: go_terms_in_cluster.append(single) 
        go_terms_in_cluster = set(go_terms_in_cluster)
        #print(go_terms_in_cluster)

        cluster_to_go_term[cluster_id] = go_terms_in_cluster

    return cluster_to_go_term   
    

In [78]:
birch_cluster_2_goterms = get_go_terms_for_clustering(df_all_merged_birch_data)
birch_cluster_2_goterms

{0: {'GO:0000272',
  'GO:0003677',
  'GO:0003700',
  'GO:0004190',
  'GO:0004252',
  'GO:0004601',
  'GO:0005509',
  'GO:0005524',
  'GO:0006508',
  'GO:0007155',
  'GO:0009055',
  'GO:0009306',
  'GO:0016020',
  'GO:0016021',
  'GO:0016887',
  'GO:0016987',
  'GO:0020037',
  'GO:0022900',
  'GO:0035438',
  'GO:0046872',
  'GO:0098869',
  'GO:2000142'},
 1: {'GO:0003824',
  'GO:0004410',
  'GO:0004792',
  'GO:0005524',
  'GO:0005737',
  'GO:0005886',
  'GO:0006535',
  'GO:0009001',
  'GO:0009055',
  'GO:0009108',
  'GO:0009399',
  'GO:0010181',
  'GO:0016021',
  'GO:0016163',
  'GO:0016612',
  'GO:0016787',
  'GO:0018697',
  'GO:0019752',
  'GO:0022900',
  'GO:0030151',
  'GO:0046872',
  'GO:0051188',
  'GO:0051536',
  'GO:0051537',
  'GO:0051539',
  'GO:0055085',
  'GO:0055114'},
 2: {'GO:0003677',
  'GO:0004252',
  'GO:0004803',
  'GO:0006260',
  'GO:0006281',
  'GO:0006313',
  'GO:0006355',
  'GO:0006508',
  'GO:0009432',
  'GO:0015074',
  'GO:0016020',
  'GO:0016021',
  'GO:0016779

In [79]:
kmeans_cluster_2_goterms = get_go_terms_for_clustering(df_all_merged_kmeans_data)
kmeans_cluster_2_goterms

{0: {'GO:0000027',
  'GO:0000036',
  'GO:0000049',
  'GO:0000155',
  'GO:0000160',
  'GO:0000166',
  'GO:0000175',
  'GO:0000287',
  'GO:0000413',
  'GO:0001123',
  'GO:0001522',
  'GO:0001682',
  'GO:0002935',
  'GO:0003674',
  'GO:0003676',
  'GO:0003677',
  'GO:0003700',
  'GO:0003723',
  'GO:0003735',
  'GO:0003746',
  'GO:0003755',
  'GO:0003849',
  'GO:0003852',
  'GO:0003883',
  'GO:0003924',
  'GO:0003992',
  'GO:0004019',
  'GO:0004055',
  'GO:0004222',
  'GO:0004316',
  'GO:0004386',
  'GO:0004397',
  'GO:0004413',
  'GO:0004519',
  'GO:0004521',
  'GO:0004526',
  'GO:0004595',
  'GO:0004638',
  'GO:0004749',
  'GO:0004810',
  'GO:0004824',
  'GO:0004825',
  'GO:0004826',
  'GO:0004829',
  'GO:0004851',
  'GO:0005515',
  'GO:0005524',
  'GO:0005525',
  'GO:0005575',
  'GO:0005737',
  'GO:0005840',
  'GO:0005886',
  'GO:0006015',
  'GO:0006164',
  'GO:0006189',
  'GO:0006281',
  'GO:0006310',
  'GO:0006355',
  'GO:0006364',
  'GO:0006400',
  'GO:0006412',
  'GO:0006414',
  'GO

# Applying GOATOOLS

- Working loosely from the goatools documentation example of using their library for exploration of gene ontology data for a given set of GO Terms

## Practice with Example

In [5]:
from goatools.base import get_godag
godag = get_godag("go-basic.obo")

  EXISTS: go-basic.obo
go-basic.obo: fmt(1.2) rel(2022-01-13) 47,158 GO Terms


In [81]:
type(godag)

goatools.obo_parser.GODag

In [6]:
from goatools.gosubdag.gosubdag import GoSubDag

In [7]:
DESC = 'methane'
NSPC = 'cellular_component'

def chooser(goterm):
    b_match = DESC in goterm.name
    
    b_leaf = not goterm.children
    
    b_nspc = True #goterm.namespace == NSPC
    
    return b_match and b_leaf and b_nspc

# Set of go_terms!
go_ids_selected = set(o.item_id for o in godag.values() if chooser(o))

print('{N} {desc} GO Terms'.format(N= len(go_ids_selected), desc = DESC))

44 methane GO Terms


In [8]:
gosubdag = GoSubDag(go_ids_selected, godag)

 GoSubDag:  44 sources in 189 GOs rcnt(True). 0 alt GO IDs
 GoSubDag: namedtuple fields: NS level depth GO alt GO_name dcnt D1 id
 GoSubDag: relationships: set()


In [9]:
go_id, go_term = max(gosubdag.go2obj.items(), key = lambda t: t[1].depth)
print(go_id, go_term.name)

GO:0018160 peptidyl-pyrromethane cofactor linkage


In [10]:
go_ids_chosen = go_term.get_all_parents()

print('{N} ancestors for {GO} "{name}"'.format(N = len(go_ids_chosen), GO = go_term.item_id, name = go_term.name))

17 ancestors for GO:0018160 "peptidyl-pyrromethane cofactor linkage"


In [11]:
go_ids_chosen.add(go_id)
nts = [gosubdag.go2nt[go] for go in go_ids_chosen]

In [12]:
fmt_str = '{I:2}) {NS} {GO:10} {dcnt:11}        D{depth:02}  {GO_name}'
print('IDX NS GO ID      Descendants Count Depth Name')
print('--- -- ---------- ----------------- ----- ------------------')
for idx, nt_go in enumerate(sorted(nts, key=lambda nt: nt.depth), 1):
    print(fmt_str.format(I=idx, **nt_go._asdict()))

IDX NS GO ID      Descendants Count Depth Name
--- -- ---------- ----------------- ----- ------------------
 1) BP GO:0008150       28427        D00  biological_process
 2) BP GO:0009987       11907        D01  cellular process
 3) BP GO:0008152        6296        D01  metabolic process
 4) BP GO:0071704        5965        D02  organic substance metabolic process
 5) BP GO:0044238        3948        D02  primary metabolic process
 6) BP GO:0044237        5474        D02  cellular metabolic process
 7) BP GO:0006807        3737        D02  nitrogen compound metabolic process
 8) BP GO:1901564        2761        D03  organonitrogen compound metabolic process
 9) BP GO:0043170        2279        D03  macromolecule metabolic process
10) BP GO:0044260        1556        D04  cellular macromolecule metabolic process
11) BP GO:0019538        1070        D04  protein metabolic process
12) BP GO:0043412        1008        D04  macromolecule modification
13) BP GO:0044267         900        D05 

## Practice with Real Sample

In [13]:
# SAMPLE => Loci: EQU24_RS22135

loci_of_interest = 'EQU24_RS22135'
loci_data_row = df_all_merged_birch_data[df_all_merged_birch_data['locus_tag'] == loci_of_interest]
go_terms_of_interest = loci_data_row['go_terms'].values[0]

go_terms_of_interest

['GO:0003924',
 'GO:0005515',
 'GO:0005525',
 'GO:0005737',
 'GO:0006400',
 'GO:0046872']

In [14]:
pd.set_option('display.max_colwidth', None)

loci_data_row['desc_string']

4208    EQU24_RS22135|mnmE|tRNA uridine-5-carboxymethylaminomethyl(34) synthesis GTPase MnmE
Name: desc_string, dtype: object

In [15]:
gosubdag = GoSubDag(go_terms_of_interest, godag)

 GoSubDag:   6 sources in  53 GOs rcnt(True). 0 alt GO IDs
 GoSubDag: namedtuple fields: NS level depth GO alt GO_name dcnt D1 id
 GoSubDag: relationships: set()


In [16]:
go_id, go_term = max(gosubdag.go2obj.items(), key = lambda t: t[1].depth)
print(go_id, go_term.name)

GO:0006400 tRNA modification


In [17]:
go_ids_chosen = go_term.get_all_parents()

print('{N} ancestors for {GO} "{name}"'.format(N = len(go_ids_chosen), GO = go_term.item_id, name = go_term.name))

22 ancestors for GO:0006400 "tRNA modification"


In [18]:
go_ids_chosen.add(go_id)
nts = [gosubdag.go2nt[go] for go in go_ids_chosen]

In [19]:
fmt_str = '{I:2}) {NS} {GO:10} {dcnt:11}        D{depth:02}  {GO_name}'
print('IDX NS GO ID      Descendants Count Depth Name')
print('--- -- ---------- ----------------- ----- ------------------')
for idx, nt_go in enumerate(sorted(nts, key=lambda nt: nt.depth), 1):
    print(fmt_str.format(I=idx, **nt_go._asdict()))

IDX NS GO ID      Descendants Count Depth Name
--- -- ---------- ----------------- ----- ------------------
 1) BP GO:0008150       28427        D00  biological_process
 2) BP GO:0009987       11907        D01  cellular process
 3) BP GO:0008152        6296        D01  metabolic process
 4) BP GO:0071704        5965        D02  organic substance metabolic process
 5) BP GO:0044238        3948        D02  primary metabolic process
 6) BP GO:0044237        5474        D02  cellular metabolic process
 7) BP GO:0006807        3737        D02  nitrogen compound metabolic process
 8) BP GO:0034641        2038        D03  cellular nitrogen compound metabolic process
 9) BP GO:1901360        2475        D03  organic cyclic compound metabolic process
10) BP GO:0006725        2094        D03  cellular aromatic compound metabolic process
11) BP GO:0046483        2025        D03  heterocycle metabolic process
12) BP GO:0043170        2279        D03  macromolecule metabolic process
13) BP GO:00061

## Practice: Using Cluster-Wide Data to Seek Enrichment

- Applying cluster 5 (almost all repressible copper loci and smo) from Kmeans Clustering

In [85]:
cluster_5 = kmeans_cluster_2_goterms[5]